In [ ]:
https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
https://www.researchgate.net/publication/336431484_An_LSTM_Short-Term_Solar_Irradiance_Forecasting_under_Complicated_Weather_Conditions/figures?lo=1

### Singular REST Request

In [ ]:
import requests
import urllib.parse

url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?"

# Define query parameters
params = {
    "api_key": "oULn1WdbouhNP5445Mbr5RIZzm10NbtzkwL5aOe3",
    "email": "justin.xu@mail.utoronto.ca",
    "wkt": "POINT(-108.5449+40.5137)",
    "names": 2014,
    "interval": 30,
}
query_string = urllib.parse.urlencode(params, safe='@()+')
complete_url = url + query_string

# Make GET request
response = requests.get(complete_url)

# Check if request was successful
if response.status_code == 200:
    print("Request Successful.")
    data = response.content.decode('utf-8')
else:
    print(f"Error: {response.status_code} -> could not retrieve data from API.")

Request Successful.


In [ ]:
columns = "Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed Units,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version"
columns = columns.split(',')

In [ ]:
from io import StringIO
import pandas as pd

df = pd.read_csv(StringIO(data), sep=",", header=None,names=columns)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
1,NSRDB,155474,-,-,-,40.53,-108.54,0,2133,-7,...,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.0.6
2,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,1,0,0,0,0,0,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,1,0,30,0,0,0,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17518,2014,12,31,21,30,-8,58,915,362,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17519,2014,12,31,22,0,-8,53,865,292,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17520,2014,12,31,22,30,-9,45,791,213,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17521,2014,12,31,23,0,-11,35,672,129,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Pandas Requests

In [ ]:
def download(url, output_path):
    df = pd.read_csv(url, skiprows=2).dropna(axis=1)
    df.insert(
        0, 'Date',
        pd.to_datetime(
            df.loc[:, ['Year', 'Month', 'Day', 'Hour', 'Minute']],
            format='%Y-%m-%d %H:%M'
        )
    )
    df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
    df = df[df.loc[:, 'GHI'] > 0]
    df.to_csv(output_path, index=False)

In [ ]:
url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?"

# Define query parameters
params = {
    "api_key": "oULn1WdbouhNP5445Mbr5RIZzm10NbtzkwL5aOe3",
    "email": "justin.xu@mail.utoronto.ca",
    "wkt": None,
    "names": None,
    "interval": 30,
}

In [ ]:
main_path = '/content'

train_years = [1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008]
test_years = [2009, 2010]

all_years = train_years + test_years

# NYC
longitude = 40.730610
latitude = -73.935242

In [ ]:
import os
import urllib.parse

location = f"POINT({latitude}+{longitude})"

data_path = os.path.join(main_path, 'data')
if not os.path.isdir(data_path):
    os.mkdir(data_path)

for each_year in all_years:
    params['wkt'] = location
    params['names'] = each_year
    query_string = urllib.parse.urlencode(params, safe='@()+')
    complete_url = url + query_string

    download(complete_url, os.path.join(data_path, f"{each_year}.csv"))
    print(f"{each_year}'s data downloaded to .csv successfully.")

### Data Exploration

In [ ]:
import pandas as pd

pd.read_csv('/content/2014.csv')

,Date,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,DHI,DNI,Fill Flag,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm)
0,2014-01-01 13:00:00,-3,35,300,64,7,-7,36,166,4,51,68.81,84.49,0.103,1020,0.786,282.6,4.5,0.0000,0.0000
1,2014-01-01 13:30:00,-2,54,464,134,7,-7,56,308,0,109,63.90,80.11,0.103,1020,0.805,282.6,4.4,0.0000,0.0000
2,2014-01-01 14:00:00,-1,70,562,205,4,-7,90,212,0,141,64.90,76.11,0.103,1020,0.825,281.8,4.4,6.8314,5.2393
3,2014-01-01 14:30:00,0,80,636,271,4,-7,112,274,0,195,60.31,72.54,0.103,1020,0.846,281.8,4.3,9.7525,7.5256
4,2014-01-01 15:00:00,0,89,684,329,4,-7,147,253,4,236,62.12,69.48,0.103,1020,0.868,278.1,4.3,12.1377,9.4053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8630,2014-12-31 19:00:00,0,67,815,349,4,-8,115,437,4,267,51.13,69.72,0.105,1020,0.346,288.0,4.7,13.7029,10.6151
8631,2014-12-31 19:30:00,0,61,768,288,4,-8,97,416,4,220,51.12,72.81,0.105,1020,0.341,288.0,4.5,10.9751,8.4655
8632,2014-12-31 20:00:00,0,53,701,218,4,-8,72,401,0,167,56.78,76.40,0.105,1020,0.338,283.7,4.3,8.0731,6.1883
8633,2014-12-31 20:30:00,0,43,594,142,4,-8,58,303,4,108,56.78,80.42,0.105,1020,0.336,283.7,4.1,0.0000,0.0000


### Dataloaders

In [1]:
import json
import os

import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler


class CSVFileNotFound(Exception):

    def __init__(self, filename, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filename = filename

    def __str__(self):
        return f'The path {self.filename} does not exist'


# TODO: what happens when that date does not exist
def reshape_data(df, start_date, end_date):
    if start_date and end_date:
        df = df.set_index('Date')[start_date:end_date]
    elif start_date:
        df = df.set_index('Date')[start_date:]
    elif end_date:
        df = df.set_index('Date')[:end_date]
    return df.index, df.loc[:, 'GHI'].values.reshape(-1, 1)


def norm_save_scaler(df, model_name, start_date, end_date):
    dates, data = reshape_data(df, start_date, end_date)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    joblib.dump(scaler, f'/content/{model_name}.gz')
    return dates, scaled_data


def norm_use_scaler(df, model_name, start_date, end_date):
    dates, data = reshape_data(df, start_date, end_date)
    scaler = joblib.load(f'/content/{model_name}.gz')
    return dates, scaler.transform(data)


def inverse_norm(data, model_name):
    scaler = joblib.load(f'/content/{model_name}.gz')
    return scaler.inverse_transform(np.array(data).reshape(-1, 1))


def concat_csv(lat, lon, years):
    df = pd.DataFrame(columns=['Date', 'GHI'])
    for year in years.split(','):
        path = f'/content/{year}.csv'
        if not os.path.isfile(path):
            raise CSVFileNotFound(path)
        df = pd.concat([df, pd.read_csv(f'/content/{year}.csv', parse_dates=True)])
    return df


def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


def create_dataloader(scaled_data, seq_length, batch_size):
    x, y = create_sequences(scaled_data, seq_length)
    dataset = torch.utils.data.TensorDataset(
        torch.from_numpy(x).float(),
        torch.from_numpy(y).float()
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2,
        shuffle=False,
        pin_memory=True
    )
    return dataloader


def dataloader(
    lat, lon, years, seq_length, batch_size, model_name, norm_callback,
    start_date, end_date
):
    df = concat_csv(lat, lon, years)
    dates, scaled_data = norm_callback(df, model_name, start_date, end_date)
    dataloader = create_dataloader(scaled_data, seq_length, batch_size)
    return dates, dataloader

### Model (LSTM)

In [2]:
import torch
import torch.nn as nn


class LSTMDrop(nn.Module):

    def __init__(self, hidden_size, num_layers, dropout, device):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.device = device
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc1 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_size, device=self.device
        )
        c0 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_size, device=self.device
        )
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc1(out[:, -1, :])
        return out

### Train

In [3]:
import torch


def train(
    lat, lon, train_years, seq_length, batch_size, model_name, start_date,
    end_date, hidden_size, num_layers, dropout, epochs, lr, decay, step_size,
    gamma
):
    _, train_loader = dataloader(
        lat, lon, train_years, seq_length, batch_size, model_name,
        norm_save_scaler, start_date, end_date
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LSTMDrop(hidden_size, num_layers, dropout, device).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=step_size, gamma=gamma
    )
    print('Start training...')
    model.train()
    for epoch in range(epochs):
        training_loss = 0.0
        for batch_id, (train_x, train_y) in enumerate(train_loader):
            train_x, train_y = train_x.to(device), train_y.to(device)
            optimizer.zero_grad()
            outputs = model(train_x)
            loss = criterion(outputs, train_y)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        scheduler.step()
        print(
            f'Epoch Summary --> Epoch: {epoch + 1}, '
            f'Epoch Loss: {training_loss / len(train_loader)}, '
            f'LR: {optimizer.param_groups[0]["lr"]}'
        )
    print('Training complete!')
    torch.save(
        model.state_dict(), (f'/content/{model_name}.pt')
    )

### Evaluate

In [4]:
import torch


def evaluate(
    lat, lon, test_years, seq_length, model_name, start_date, end_date,
    hidden_size, num_layers, **kwargs
):
    DROPOUT, BATCH_SIZE = 0, 1
    dates, test_loader = dataloader(
        lat, lon, test_years, seq_length, BATCH_SIZE, model_name,
        norm_use_scaler, start_date, end_date
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LSTMDrop(hidden_size, num_layers, DROPOUT, device).to(device)
    model.load_state_dict(
        torch.load(f'/content/{model_name}.pt', map_location=device)
    )
    criterion = torch.nn.MSELoss()
    print('Start testing...')
    model.eval()
    testing_loss = 0.0
    predicted = []
    actual = []
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            outputs = model(x)
            predicted.append(outputs.item())
            actual.append(y.item())
            loss = torch.sqrt(criterion(outputs, y))
            testing_loss += loss.item()
    rmse = testing_loss/len(test_loader)
    print(f'RMSE: {rmse}')
    print('Done testing!')
    return (
        dates[seq_length+1:],
        inverse_norm(predicted, model_name),
        inverse_norm(actual, model_name),
        rmse
    )

### Plotly -> Look into Dash

In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


def _construct_df(dates, predicted, actual):
    df_dict = {
        'Dates': dates.to_numpy(),
        'Predicted': predicted.flatten(),
        'Actual': actual.flatten()
    }
    return pd.DataFrame(df_dict)


def pretty_plot(dates, predicted, actual, rmse):
    df = _construct_df(dates, predicted, actual)
    layout = go.Layout(
        title=dict(
            text=f'Predicted vs Actual Irradiance (RMSE: {rmse})', x=0.5
        ),
        font=dict(size=18),
        xaxis=dict(showgrid=False, title='Date'),
        yaxis=dict(showgrid=False, zeroline=False, title='Clearsky GHI (W/m^2)'),
        colorway=px.colors.qualitative.Plotly
    )
    fig = go.Figure(layout=layout)
    fig.add_trace(
        go.Scatter(
            x=df['Dates'], y=df['Actual'], name='Actual', line=dict(width=3)
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Dates'],
            y=df['Predicted'],
            mode='lines',
            name='Predicted',
            line=dict(width=3)
        )
    )
    fig.show()

### Train

In [5]:
# NYC
lon = 40.730610
lat = -73.935242

train_years = '2010,2011,2012,2013,2014,2015,2016,2017,2018,2019'
test_years = '2020'
start_splice = None
end_splice = None

horizon = 5
interval = 30

model = 'LSTM_Drop'
batch_size = 5
hidden_layer_size = 35
num_layers = 2
dropout = 0.3
epochs = 10
learning_rate = 0.001
weight_decay = 0.0001
decay_step_size = 2
learning_rate_decay = 0.5

print(f'Now training a {model} with dropout...')

train(lat, lon, train_years, horizon, batch_size, model, start_splice,
    end_splice, hidden_layer_size, num_layers, dropout, epochs, learning_rate, weight_decay, decay_step_size,
    learning_rate_decay)

Now training a LSTM_Drop with dropout...
Start training...
Epoch Summary --> Epoch: 1, Epoch Loss: 0.008742284956221855, LR: 0.001
Epoch Summary --> Epoch: 2, Epoch Loss: 0.00737135890024744, LR: 0.0005
Epoch Summary --> Epoch: 3, Epoch Loss: 0.0069708534353931885, LR: 0.0005
Epoch Summary --> Epoch: 4, Epoch Loss: 0.0069606212116008655, LR: 0.00025
Epoch Summary --> Epoch: 5, Epoch Loss: 0.00684683937223815, LR: 0.00025
Epoch Summary --> Epoch: 6, Epoch Loss: 0.006822735236801221, LR: 0.000125
Epoch Summary --> Epoch: 7, Epoch Loss: 0.0067834218754550995, LR: 0.000125
Epoch Summary --> Epoch: 8, Epoch Loss: 0.006781115022184749, LR: 6.25e-05
Epoch Summary --> Epoch: 9, Epoch Loss: 0.006754753785883573, LR: 6.25e-05
Epoch Summary --> Epoch: 10, Epoch Loss: 0.006735747783399034, LR: 3.125e-05
Training complete!


### Evaluate

In [6]:
dates, predicted, actual, rmse = evaluate(lat, lon, test_years, horizon, model, start_splice,
    end_splice, hidden_layer_size, num_layers)

Start testing...
RMSE: 0.043954216976716746
Done testing!


In [9]:
pretty_plot(dates, predicted, actual, round(rmse, 2))

### ArgParse


In [ ]:
def main():
    parser = argparse.ArgumentParser(
        description='Download train and test data from NSRDB'
    )
    parser.add_argument(
        '--lat',
        type=float,
        help=
        'Latitude (to avoid errors make sure this value is within the continental United States) [Required]',
        required=True
    )
    parser.add_argument(
        '--lon',
        type=float,
        help=
        'Longitude (to avoid errors make sure this value is within the continental United States) [Required]',
        required=True
    )
    parser.add_argument(
        '--train-years',
        type=str,
        help=(
            'Comma seperated value string with years to download training data \
        from (1998-2017 according to the official NSRDB docs) [Required]'
        ),
        required=True
    )
    parser.add_argument(
        '--test-years',
        type=str,
        help=(
            'Comma seperated value string with years to download testing data \
        from (1998-2017 according to the official NSRDB docs) [Required]'
        ),
        required=True
    )
    parser.add_argument(
        '--interval',
        type=str,
        default='30',
        help='30 or 60 minute interval data [default: 30]'
    )
    args = vars(parser.parse_args())

    download_all(**args)